This notebook is devoted to a template for HOG algorithm. After the template implementation, there is also one working example how to use the template. In this example, you can also change the used parameters.

For this template, you can specify three function. They have to follow some rules though:
  - get_directions_and_magnitudes takes an image and k_size, it is expected to count the directions and magnitues of gradients in the image. For each pixel, it is expect to count them using k_size as a neighborhood size and return these two as a tuple. The return value is completely up to you though, just keep in mind that the output shall be used as an input for directions_and_magnitudes_to_cells
  - directions_and_magnitudes_to_cells takes the result of get_directions_and_magnitudes and k_size as an input. It is expected to split the result into cells of k_size. The output of this function should be a tuple of two - cells of directions and magnitudes. Based on these, the final histogram shall be counted.
  - get_histogram takes a direction_cell, magnitude_cell, hist_bins and should count a histogram (or any other descriptor) for this cell. For the sake of histogram, it takes also hist_bins so it can be specified how big is the histogram suppopsed to be.

In [1]:
from collections.abc import Callable
import numpy as np

class HOG:
    
    def __init__(self, 
                get_directions_and_magnitudes: Callable,
                directions_and_magnitudes_to_cells: Callable,
                get_histogram: Callable
                ):
        self._get_directions_and_magnitudes = get_directions_and_magnitudes
        self._directions_and_magnitudes_to_cells = directions_and_magnitudes_to_cells
        self._get_histogram = get_histogram        
    
    def get_feature_vector(self, image: np.ndarray, cell_size: int=8, hist_bins: int=9, k_size: int=3) -> np.ndarray:
        
        # get directions and magnitues from the image using your provided fucntion
        processed_dir_and_mag = self._get_directions_and_magnitudes(image, k_size)
        # split the result of your processing to the cells, the expected output are different cells for directions and magnitudes
        direction_cells, magnitude_cells = self._directions_and_magnitudes_to_cells(processed_dir_and_mag, cell_size)
        
        hists = []
        for cell_i in range(len(direction_cells)):
            hists.append(self._get_histogram(direction_cells[cell_i], magnitude_cells[cell_i], hist_bins))
        
        return np.array(hists).flatten()
        
        

Let's take a look at that working example. We provide implementation for all the functions needed in the template. Feel free to adjust the parameters of functions and see the effects.

In [2]:
import cv2
def get_directions_and_magnitudes(image, k_size):
    # Calculate the gradient in the x-direction
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=k_size)

    # Calculate the gradient in the y-direction
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=k_size)

    # Calculate the magnitude of the gradient
    magnitudes = np.sqrt(sobel_x ** 2 + sobel_y ** 2)

    # Calculate the direction of the gradient
    directions = np.arctan2(sobel_y, sobel_x)

    return directions, magnitudes

In [3]:
def directions_and_magnitudes_to_cells(processed_dir_and_mag, cell_size):
    directions, magnitudes = processed_dir_and_mag
    direction_cells = []
    magnitude_cells = []

    for i in range(0, directions.shape[0], cell_size):
        for j in range(0, directions.shape[1], cell_size):
            direction_cells.append(directions[i:i + cell_size, j:j + cell_size])
            magnitude_cells.append(magnitudes[i:i + cell_size, j:j + cell_size])

    return direction_cells, magnitude_cells

In [4]:
def get_histogram(directions, magnitudes, hist_bins=9):
    hist = np.zeros(hist_bins)

    # get histogram based on directions and magnitudes
    for i in range(directions.shape[0]):
        for j in range(directions.shape[1]):
            # get bin
            bin = int(directions[i][j] / (2 * np.pi) * hist_bins)

            # add magnitude to bin
            hist[bin] += magnitudes[i][j]

    return hist

In [10]:
from PIL import Image

hog = HOG(get_directions_and_magnitudes, directions_and_magnitudes_to_cells, get_histogram)
# load image and convert it to gray scale and a shape 128x128 
img = Image.open("test_img.png").convert('L')
img = np.array(img)

hog.get_feature_vector(img)

array([132.6983826 ,  71.75583961,  38.55260241, ...,   0.        ,
         8.        ,   0.        ])